In [1]:
import itertools
import logging

import click
import numpy as np
import pandas as pd
import pylandstats as pls
import salem
import xarray as xr
from tqdm import tqdm

from lausanne_greening_scenarios import settings

# register tqdm with pandas to be able to use `progress_apply`
tqdm.pandas()

HIGH_TREE_CLASS_VAL = 1
OTHER_CLASS_VAL = 2

# metrics = ('MPS', 'ED', 'MSI')
METRICS = ['area_mn', 'edge_density', 'shape_index_mn']

/home/martibosch/anaconda3/envs/lausanne-greening-scenarios/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
scenario_ds_filepath = '../data/processed/scenario-config.nc'
biophysical_table_filepath = '../data/processed/biophysical-table.csv'
shade_threshold = 0.75

In [4]:
attrs

{'pyproj_srs': 'epsg:2056'}

In [5]:
scenario_ds = xr.open_dataset(scenario_ds_filepath)
scenario_lulc_da = scenario_ds['LULC']

scenario_dims = scenario_lulc_da.coords.dims[:2]
res = scenario_ds.salem.grid.dx
nodata = scenario_lulc_da.attrs['nodata']
change_props = scenario_lulc_da['change_prop'].values.copy()
change_props.sort()

biophysical_df = pd.read_csv(biophysical_table_filepath)

In [6]:
# define the functions so that the fixed arguments are curried into them,
# except for `metrics`
def compute_metrics(row, metrics):
    # landscape_arr = sg.generate_landscape_arr(shade_threshold,
    #                                           row['change_prop'],
    #                                           interaction=row['interaction'])
    lulc_arr = scenario_lulc_da.sel({
        scenario_dim: row[scenario_dim]
        for scenario_dim in scenario_dims
    }).values
    landscape_arr = np.full_like(lulc_arr, nodata)
    landscape_arr[lulc_arr != nodata] = OTHER_CLASS_VAL
    landscape_arr[np.isin(
        lulc_arr,
        biophysical_df[biophysical_df['shade'] >= shade_threshold]
        ['lucode'])] = HIGH_TREE_CLASS_VAL
    ls = pls.Landscape(landscape_arr, (res, res), nodata)
    # return [
    #     getattr(ls, metric)(high_tree_class_val) for metric in metrics]
    return pd.Series({
        metric: getattr(ls, metric)(HIGH_TREE_CLASS_VAL)
        for metric in metrics
    })

def compute_endpoint_metrics(row, metrics):
    # interaction could be anything, since we are changing none or all the
    # changeable pixels
    _row = dict(change_prop=row['change_prop'], interaction='cluster')
    return compute_metrics(_row, metrics)

# prepare the dataframe of metrics (except PLAND) for each scenario,
# except for the endpoints (change proportion of 0 and 1, since these will
# be the same for the cluster/scatter interactions)
scenario_df = pd.DataFrame(list(
    itertools.product(scenario_lulc_da['interaction'].values,
                      change_props[1:-1])),
                           columns=['interaction', 'change_prop'])
for metric in METRICS:
    scenario_df[metric] = np.nan
# now fill it by computing the landscape metrics
scenario_df[METRICS] = scenario_df.progress_apply(compute_metrics,
                                                  axis=1,
                                                  args=(METRICS, ))


  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:08,  1.00it/s]

RuntimeError: structure and input must have equal rank

In [7]:
scenario_df

,interaction,change_prop,area_mn,edge_density,shape_index_mn
0,cluster,0.250,NaN,NaN,NaN
1,cluster,0.375,NaN,NaN,NaN
2,cluster,0.500,NaN,NaN,NaN
3,cluster,0.625,NaN,NaN,NaN
4,cluster,0.750,NaN,NaN,NaN
5,scatter,0.250,NaN,NaN,NaN
6,scatter,0.375,NaN,NaN,NaN
7,scatter,0.500,NaN,NaN,NaN
8,scatter,0.625,NaN,NaN,NaN
9,scatter,0.750,NaN,NaN,NaN
